In [1]:
# Tutorial: https://pytorch.org/tutorials/beginner/fgsm_tutorial.html
# FGSM: perturbed_image=image+epsilon∗sign(data_grad)=x+ϵ∗sign(∇ J(θ,x,y))

In [2]:
# import
import torch
import wandb
from fastai.callback.wandb import WandbCallback
from fastai.data.all import (
    CategoryBlock,
    DataBlock,
    DataLoaders,
    RandomSplitter,
    RegressionBlock,
)

import fastai
from fastai.vision.all import *
from fastai.losses import CrossEntropyLossFlat
from fastai.vision.data import ImageBlock, ImageDataLoaders
from fastai.vision.utils import get_image_files
from torch import nn
import torch.optim as optim
from fastai.vision.models import resnet18
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
from math import radians

In [3]:
# import attacks
from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.torch.attacks.projected_gradient_descent import (
    projected_gradient_descent,
)
#from cleverhans.torch.attacks.carlini_wagner_l2 import carlini_wagner_l2
from carlini_wagner_l2 import carlini_wagner_l2

In [4]:
fastai.__version__

'2.7.11'

In [5]:
# Assign GPU
torch.cuda.set_device(2)

print("Running on GPU: " + str(torch.cuda.current_device()))

Running on GPU: 2


In [6]:
# start a new wandb run to track this script
run = wandb.init(
        project="DataAugmentation",
        entity="arcslaboratory",
        notes="Adversarial Training"
    )

wandb: Currently logged in as: christymnm (arcslaboratory). Use `wandb login --relogin` to force relogin


cat: /sys/module/amdgpu/initstate: No such file or directory
ERROR:root:Driver not initialized (amdgpu not found in modules)


In [7]:
def y_from_filename(rotation_threshold, filename) -> str:
    """
    Extracts the direction label from the filename of an image.

    Example: "path/to/file/001_000011_-1p50.png" --> "right"
    """
    filename_stem = Path(filename).stem
    angle = float(filename_stem.split("_")[2].replace("p", "."))

    if angle > rotation_threshold:
        return "left"
    elif angle < -rotation_threshold:
        return "right"
    else:
        return "forward"
        
def get_dls(data_path: str):
    # NOTE: not allowed to add a type annotation to the input

    image_filenames: list = get_image_files(data_path)  # type:ignore

    # Using a partial function to set the rotation_threshold from args
    label_func = partial(y_from_filename, radians(5)) # radians(5) = 0.0872665

    return ImageDataLoaders.from_name_func(
            data_path,
            image_filenames,
            label_func,
            valid_pct=0.2,
            bs=64)

In [8]:
# get data
data_folder = "/home/cemb2020/scr2023/data/WanderingStaticTextures/"
# get model
models_folder = "/data/clark/scr2023/data/WanderingStaticTextures/models/"
model_name = "wandering-static_rep00.pkl"

In [9]:
learner = load_learner(models_folder + model_name, cpu=False)

In [10]:
epsilons = [.05]

In [11]:
def test(model, test_loader, epsilon):

    # save right filename f"000_{i:0>{6}}_-1p00.png"
    # save left filename f"000_{i:0>{6}}_1p00.png"
    # save forward filename f"000_{i:0>{6}}_0p00.png"
    
    # Accuracy counter
    fgsm_correct = 0
    pgd_correct = 0
    cw_correct = 0
    
    fgsm_adv_examples = []
    pgd_adv_examples = []
    cw_adv_examples = []

    clean_examples = []
    
    # Loop over all examples in test set
    for i, (data, target) in enumerate(test_loader):
        
        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True
        
        # Forward pass the data through the model
        output = learner.model(data)
            
        init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        
        # If the initial prediction is wrong, don't bother attacking, just move on
        if init_pred.item() != target.item():
            continue

        # Get adversarial examples
        #fgsm_ex = fast_gradient_method(learner.model, data, epsilon, np.inf, clip_min=-1, clip_max=1)
        #pgd_ex = projected_gradient_descent(learner.model, data, epsilon, 0.005, 40, np.inf, clip_min=-1, clip_max=1, sanity_checks=False)
        cw_ex = carlini_wagner_l2(learner.model, data, n_classes=3, clip_min=-1)
        
        # Re-classify the perturbed image
        fgsm_out = learner.model(fgsm_ex)
        pgd_out = learner.model(pgd_ex)
        cw_out = learner.model(cw_ex)
        
        # Check for success
        fgsm_final_pred = fgsm_out.max(1, keepdim=True)[1] # get the index of the max log-probability
        pgd_final_pred = pgd_out.max(1, keepdim=True)[1]
        cw_final_pred = cw_out.max(1, keepdim=True)[1]

        # collect clean examples
        if len(clean_examples) < 5:
                ex = data.squeeze().detach().cpu().numpy()
                clean_examples.append((init_pred.item(), target.item(), ex))

        # keep count of adv examples and save 5 of each
        if fgsm_final_pred.item() == target.item():
            fgsm_correct += 1
        else:
            # Save some adv examples for visualization later
            if len(fgsm_adv_examples) < 5:
                adv_ex = fgsm_ex.squeeze().detach().cpu().numpy()
                fgsm_adv_examples.append((init_pred.item(), fgsm_final_pred.item(), adv_ex))    
        if pgd_final_pred.item() == target.item():
            pgd_correct += 1
        else:
            # Save some adv examples for visualization later
            if len(pgd_adv_examples) < 5:
                adv_ex = pgd_ex.squeeze().detach().cpu().numpy()
                pgd_adv_examples.append( (init_pred.item(), pgd_final_pred.item(), adv_ex) )
        if cw_final_pred.item() == target.item():
            cw_correct += 1
        else:
            # Save some adv examples for visualization later
            if len(cw_adv_examples) < 5:
                adv_ex = cw_ex.squeeze().detach().cpu().numpy()
                cw_adv_examples.append( (init_pred.item(), cw_final_pred.item(), adv_ex) )

    # Calculate final accuracy for this epsilon
    fgsm_acc = fgsm_correct/float(len(test_loader))
    pgd_acc = pgd_correct/float(len(test_loader))
    cw_acc = cw_correct/float(len(test_loader))
    
    print(f"Epsilon: {epsilon}\tFGSM Accuracy = {fgsm_correct} / {len(test_loader)} = {fgsm_acc}")
    print(f"Epsilon: {epsilon}\tPGD Accuracy = {pgd_correct} / {len(test_loader)} = {pgd_acc}")
    print(f"CW Accuracy = {cw_correct} / {len(test_loader)} = {cw_acc}")

    # Return the accuracy and an adversarial example
    return clean_examples, fgsm_adv_examples, pgd_adv_examples, cw_adv_examples

In [12]:
accuracies = []
clean_examples, fgsm_examples, pgd_examples, cw_examples = [], [], [], []

image_filenames: list = get_image_files(data_folder)
test_loader = learner.dls.test_dl(image_filenames, bs=1, with_labels=True)
print(len(test_loader))

# Run test for each epsilon
for eps in epsilons:
    clean_ex, fgsm_ex, pgd_ex, cw_ex = test(learner, test_loader, eps)
    clean_examples.append(clean_ex)
    fgsm_examples.append(fgsm_ex)
    pgd_examples.append(pgd_ex)
    cw_examples.append(cw_ex)

1261
TensorImage(0.0321, device='cuda:2', grad_fn=<AliasBackward0>)
TensorImage(12.1863, device='cuda:2', grad_fn=<AliasBackward0>)


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
# Plot several examples of adversarial samples at each epsilon
cnt = 0
plt.figure(figsize=(8,10))
for i in range(len(epsilons)):
    for examples in [clean_examples, fgsm_examples, pgd_examples, cw_examples]:
        for j in range(len(examples[i])):
            cnt += 1
            plt.subplot(len(epsilons),len(examples[0]),cnt)
            plt.xticks([], [])
            plt.yticks([], [])
            if j == 0:
                plt.ylabel(f"Eps: {epsilons[i]}", fontsize=14)
            orig,adv,ex = examples[i][j]
            plt.title(f"{orig} -> {adv}")
            outmap_min = np.min(ex)
            outmap_max = np.max(ex)
            ex = (ex - outmap_min) / (outmap_max - outmap_min)
            plt.imshow((np.transpose(ex, (1, 2, 0))))
plt.tight_layout()
plt.show()

In [ ]:
run = wandb.init(
    project="DataAugmentation",
    entity="arcslaboratory",
    notes="adversarial example generation",
)

if run is None:
    raise Exception("wandb.init() failed")